In [3]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import itertools
import fnmatch
import scipy.io as sio

from subprocess import check_output
from glob import glob
from PIL import Image
from pathlib import Path

import matplotlib.pylab as plt
import matplotlib.image as mpimg

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils 
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau

sns.set(style='white', context='notebook', palette='deep')

In [5]:
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:1']

## Load Data

In [6]:
train_path = "I:\\Honours-Project\\data\\sorted\\agg\\train\\"
train_csv = "I:\\Honours-Project\\data\\sorted\\agg\\train\\ids.csv"
test_path = "I:\\Honours-Project\\data\\sorted\\agg\\test\\"
test_csv = "I:\\Honours-Project\\data\\sorted\\agg\\test\\ids.csv"

dest_path = "I:\\Honours-Project\\data\\sorted\\agg\\"

In [7]:
x_train = []
y_train = []

x_test = []
y_test = []

'''for file in os.listdir(train_path):
    if fnmatch.fnmatch(file, '*.npy'):
        x_train.append(np.load(train_path+file))

x_train = np.array(x_train)    '''    

df = pd.read_csv(train_csv)

for path in df['name']:
    x_train.append(np.load(dest_path+path+".npy"))
    
x_train = np.array(x_train)
y_train = df["flow_rate"].values

'''for file in os.listdir(test_path):
    if fnmatch.fnmatch(file, '*.npy'):
        x_test.append(np.load(test_path+file))

x_test = np.array(x_test)        '''

df = pd.read_csv(test_csv)

for path in df['name']:
    x_test.append(np.load(dest_path+path+".npy"))
    
x_test = np.array(x_test)
y_test = df["flow_rate"].values

In [8]:
nb_classes = 3

y_train[y_train == 1800] = 0
y_train[y_train == 3600] = 1
y_train[y_train == 7200] = 2

y_test[y_test == 1800] = 0
y_test[y_test == 3600] = 1
y_test[y_test == 7200] = 2

y_train = np_utils.to_categorical(y_train, nb_classes) 
y_test = np_utils.to_categorical(y_test, nb_classes)

In [9]:
print(x_test.shape)
x_test = x_test.reshape(40,991,507,1)

print(x_train.shape)
x_train = x_train.reshape(362,991,507,1)

(40, 991, 507)
(362, 991, 507)


## Run CNN

Source: https://www.kaggle.com/yassineghouzam/introduction-to-cnn-keras-0-997-top-6

In [10]:
# Set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape=x_train.shape[1:]))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
# model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation = "softmax"))

In [11]:
# Define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [12]:
# Compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [13]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [14]:
epochs = 30 # Turn epochs to 30 to get 0.9967 accuracy
batch_size = 32

In [15]:
# Fit the model
"""history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])"""
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=2, 
                    validation_data=(x_test, y_test), 
                    callbacks=[learning_rate_reduction])
score = model.evaluate(x_test, y_test, verbose=0)

Train on 362 samples, validate on 40 samples
Epoch 1/30


ResourceExhaustedError: OOM when allocating tensor with shape[1991808,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: dense_1/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=1830279, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_1/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'dense_1/random_uniform/RandomUniform', defined at:
  File "C:\Users\Duo\Anaconda3\envs\fastai\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Duo\Anaconda3\envs\fastai\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Duo\Anaconda3\envs\fastai\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Duo\Anaconda3\envs\fastai\lib\si

In [ ]:
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['acc'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_acc'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)